In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


import os


# In[17]:


os.getcwd()


# In[18]:


os.chdir('/home/pi/lab/')


# In[19]:


import csv
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# In[20]:


print(tf.__version__)


# In[21]:


from nltk.corpus import stopwords
import nltk
STOPWORDS = set(stopwords.words('english'))


# In[22]:


vocab_size = 5000
embedding_dim = 64
max_length = 200
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'
training_portion = .8


# In[23]:


articles = []
labels = []

with open("bbc-text.csv", 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    
    for row in reader:
        labels.append(row[0])
        article = row[1]
        
        for word in STOPWORDS:
            
            token = ' ' + word + ' '
            article = article.replace(token, ' ')
            article = article.replace(' ', ' ')
        articles.append(article)
        
    print(len(labels))
    print(len(articles))


# In[24]:


train_size = int(len(articles) * training_portion)

train_articles = articles[0: train_size]
train_labels = labels[0: train_size]

validation_articles = articles[train_size:]
validation_labels = labels[train_size:]

print(train_size)
print(len(train_articles))
print(len(train_labels))
print(len(validation_articles))
print(len(validation_labels))


# In[25]:


tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_articles)
word_index = tokenizer.word_index
dict(list(word_index.items())[0:10])


# In[26]:


train_sequences = tokenizer.texts_to_sequences(train_articles)
print(train_sequences[10])


# In[27]:


train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(train_sequences[0]))
print(len(train_padded[0]))

print(len(train_sequences[1]))
print(len(train_padded[1]))

print(len(train_sequences[10]))
print(len(train_padded[10]))


# In[28]:


print(train_padded[10])


# In[29]:



validation_sequences = tokenizer.texts_to_sequences(validation_articles)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

print(len(validation_sequences))
print(validation_padded.shape)


# In[30]:


label_tokenizer = Tokenizer()
label_tokenizer.fit_on_texts(labels)

training_label_seq = np.array(label_tokenizer.texts_to_sequences(train_labels))
validation_label_seq = np.array(label_tokenizer.texts_to_sequences(validation_labels))
print(training_label_seq[0])
print(training_label_seq[1])
print(training_label_seq[2])
print(training_label_seq.shape)

print(validation_label_seq[0])
print(validation_label_seq[1])
print(validation_label_seq[2])
print(validation_label_seq.shape)


# In[31]:


reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_article(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])
print(decode_article(train_padded[10]))
print('---')
print(train_articles[10])


# In[32]:


model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.summary()


# In[34]:


print(set(labels))


# In[ ]:
checkpoint_path = "cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpoint_dir = os.path.dirname(checkpoint_path)

1.14.0
2225
2225
1780
1780
1780
445
445
[2431, 1, 225, 4995, 22, 641, 587, 225, 4995, 1, 1, 1663, 1, 1, 2431, 22, 565, 1, 1, 140, 278, 1, 140, 278, 796, 823, 662, 2307, 1, 1144, 1694, 1, 1721, 4996, 1, 1, 1, 1, 1, 4738, 1, 1, 122, 4515, 1, 2, 2874, 1505, 352, 4739, 1, 52, 341, 1, 352, 2171, 3963, 41, 22, 3796, 1, 1, 1, 1, 543, 1, 1, 1, 835, 631, 2366, 347, 4740, 1, 365, 22, 1, 787, 2367, 1, 4302, 138, 10, 1, 3667, 682, 3533, 1, 22, 1, 414, 823, 662, 1, 90, 13, 633, 1, 225, 4995, 1, 600, 1, 1694, 1021, 1, 4997, 808, 1864, 117, 1, 1, 1, 2974, 22, 1, 99, 278, 1, 1607, 4998, 543, 493, 1, 1444, 4741, 778, 1320, 1, 1861, 10, 33, 641, 319, 1, 62, 478, 565, 301, 1506, 22, 479, 1, 1, 1666, 1, 797, 1, 3066, 1, 1364, 6, 1, 2431, 565, 22, 2971, 4735, 1, 1, 1, 1, 1, 850, 39, 1825, 675, 297, 26, 979, 1, 882, 22, 361, 22, 13, 301, 1506, 1342, 374, 20, 63, 883, 1096, 4303, 247]
426
200
192
200
186
200
[2431    1  225 4995   22  641  587  225 4995    1    1 1663    1    1
 2431   22  565    1    1  140

In [2]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [3]:
model.load_weights(checkpoint_path)

In [4]:
history = model.evaluate(train_padded, training_label_seq, verbose=2 )

1780/1780 - 54s - loss: 0.0649 - acc: 0.9860


In [5]:
import h5py

In [6]:
model.save('my_model.h5')

In [8]:
new = tf.keras.models.load_model('my_model.h5')

In [9]:
history = new.evaluate(train_padded, training_label_seq, verbose=2 )

1780/1780 - 58s - loss: 0.0649 - acc: 0.9860


In [16]:
res = new.predict(train_padded)

In [26]:
res.shape

(1780, 6)

In [10]:

import matplotlib.pyplot as plt

def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()

In [17]:
plot_graphs(res, "acc")

AttributeError: 'numpy.ndarray' object has no attribute 'history'

In [15]:
history

[0.06486534038035388, 0.98595506]